<img title="a title" alt="Alt text" src="social.png" width="100" height="100"> 

# Análise de Repositórios GitHub por Linguagem de Programação

## 🌍 Introdução
Este projeto tem como objetivo criar um dataset rico e estruturado com informações dos repositórios mais relevantes do GitHub, organizados pelas 10 linguagens de programação mais populares em 2025. Com esses dados, podemos responder perguntas como:

- Qual linguagem tem os projetos mais estrelados?
- Há correlação entre o número de contribuidores e a atividade do repositório?
- Quais licenças são mais comuns em projetos open-source?
- Como a localização geográfica dos donos influencia a popularidade dos repositórios?

O dataset gerado pode ser usado para:

✅ Identificação de tendências no desenvolvimento de software

✅ Análise de comunidades open-source

✅ Tomada de decisões para contribuições ou adoção de tecnologias

In [11]:
# !pip install pandas
# !pip install requests

### Imports

In [12]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import random
import os

## 🔄 Rotação de Tokens
Para evitar limites da API, usamos múltiplos tokens com seleção aleatória:

In [ ]:
TOKENS = [
    "TOKENS"
]   

def get_headers():
    return {"Authorization": f"token {random.choice(TOKENS)}"}

## 🔧 Configuração Inicial
Definir as linguagens para análise e parâmetros de paginação para extração eficiente dos dados.

In [14]:
LANGUAGES = ["Python", "JavaScript", "Java", "C#", "C++", "TypeScript", "Go", "Rust", "Kotlin", "Swift"]
PER_PAGE = 100
PAGES = 15
OUTPUT_FILE = "../dados/github_repos_completos.csv"

## 👤 Obter Informações do Dono do Repositório
- Acessa o perfil do dono via API

- Trata 3 informações principais: tipo, quantidade de repositórios e localização

- Implementa tratamento robusto de erros

In [15]:
def get_owner_info(owner_url):
    try:
        response = requests.get(owner_url, headers=get_headers())
        if response.status_code == 200:
            owner_data = response.json()
            return {
                "owner_type": owner_data.get("type", "User"),
                "owner_public_repos": owner_data.get("public_repos", 0),
                "owner_location": owner_data.get("location", None)
            }
    except Exception as e:
        print(f"Erro ao buscar owner: {e}")
    return {
        "owner_type": "User",
        "owner_public_repos": 0,
        "owner_location": None
    }

## 📈 Coletar Estatísticas do Repositório
- Paginação automática para contagem de contribuidores

- Filtro temporal para issues fechadas

- Delay entre requisições para evitar rate limits

In [16]:
def get_paginated_count(url, headers, max_pages=50):
    """Função auxiliar para contar itens com paginação completa"""
    total_count = 0
    page = 1
    
    while page <= max_pages:
        try:
            paginated_url = f"{url}?page={page}&per_page=100"
            response = requests.get(paginated_url, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                if not data or len(data) == 0:
                    break
                total_count += len(data)
                page += 1
                time.sleep(0.5)  # Delay menor entre páginas
            else:
                break
                
        except Exception as e:
            print(f"Erro na paginação: {e}")
            break
            
    return total_count

def get_repo_stats(owner, repo_name):
    stats = {
        "subscribers_count": 0,
        "last_year_commits": 0,
        "contributors": 0,
        "closed_issues": 0,
        "pull_requests": 0
    }
    
    headers = get_headers()
    
    try:
        # CORREÇÃO: Subscribers com paginação completa
        subscribers_url = f"https://api.github.com/repos/{owner}/{repo_name}/subscribers"
        stats["subscribers_count"] = get_paginated_count(subscribers_url, headers)
        
        # Commits do último ano
        participation = requests.get(
            f"https://api.github.com/repos/{owner}/{repo_name}/stats/participation",
            headers=headers
        )
        if participation.status_code == 200:
            participation_data = participation.json()
            if participation_data and "all" in participation_data:
                stats["last_year_commits"] = sum(participation_data["all"][-52:])

        # CORREÇÃO: Contributors com paginação completa
        contributors_url = f"https://api.github.com/repos/{owner}/{repo_name}/contributors"
        stats["contributors"] = get_paginated_count(contributors_url, headers)

        # CORREÇÃO: Issues fechadas nos últimos 6 meses com paginação
        since_date = (datetime.now() - timedelta(days=180)).isoformat()
        closed_issues_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues"
        
        # Para issues fechadas, precisamos usar parâmetros específicos
        page = 1
        closed_count = 0
        while page <= 20:  # Limite razoável para issues
            try:
                url = f"{closed_issues_url}?state=closed&since={since_date}&page={page}&per_page=100"
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    issues = response.json()
                    if not issues:
                        break
                    closed_count += len(issues)
                    page += 1
                    time.sleep(0.5)
                else:
                    break
            except:
                break
        stats["closed_issues"] = closed_count

        # CORREÇÃO: Pull Requests com paginação completa
        prs_url = f"https://api.github.com/repos/{owner}/{repo_name}/pulls"
        stats["pull_requests"] = get_paginated_count(prs_url, headers)

    except Exception as e:
        print(f"Erro ao buscar stats para {owner}/{repo_name}: {e}")
    
    return stats

**NOVA FUNCIONALIDADE**: Salva dados após cada linguagem
**NOVA FUNCIONALIDADE**: Append mode para não perder dados anteriores
**NOVA FUNCIONALIDADE**: Backup automático


In [17]:
def save_data_incremental(df, language, is_first_language=False):
    """Salva dados incrementalmente após cada linguagem"""
    try:
        if is_first_language and os.path.exists(OUTPUT_FILE):
            # Backup do arquivo existente
            backup_name = f"backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{OUTPUT_FILE}"
            os.rename(OUTPUT_FILE, backup_name)
            print(f"Backup criado: {backup_name}")
        
        # Salva ou adiciona ao arquivo
        if not os.path.exists(OUTPUT_FILE) or is_first_language:
            df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8', mode='w')
            print(f"Arquivo criado: {OUTPUT_FILE}")
        else:
            df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8', mode='a', header=False)
            print(f"Dados adicionados ao arquivo: {OUTPUT_FILE}")
            
        # Salva backup específico da linguagem
        lang_file = f"backup_{language.lower()}_repos.csv"
        df.to_csv(lang_file, index=False, encoding='utf-8')
        print(f"Backup da linguagem salvo: {lang_file}")
        
    except Exception as e:
        print(f"Erro ao salvar dados: {e}")

def load_existing_data():
    """Carrega dados existentes se houver"""
    if os.path.exists(OUTPUT_FILE):
        try:
            return pd.read_csv(OUTPUT_FILE)
        except:
            return pd.DataFrame()
    return pd.DataFrame()


## 🔍 Função Principal: Busca por Linguagem
- Rotação automática de tokens

- Intervalos entre requisições

- Consolidação em DataFrame

In [18]:
def fetch_repos_by_language(language, retry_count=3):
    all_repos = []
    
    print(f"\n🔍 Iniciando coleta para {language}...")
    
    for page in range(1, PAGES + 1):
        for attempt in range(retry_count):
            try:
                url = f"https://api.github.com/search/repositories?q=language:{language}&sort=stars&page={page}&per_page={PER_PAGE}"
                response = requests.get(url, headers=get_headers())
                
                if response.status_code == 403:
                    print(f"Rate limit excedido na página {page}, aguardando...")
                    time.sleep(120)  # Espera 2 minutos
                    continue
                    
                if response.status_code == 422:
                    print(f"Página {page} muito alta, parando coleta para {language}")
                    break
                    
                response.raise_for_status()
                
                repos_data = response.json()
                if "items" not in repos_data:
                    print(f"Sem repositórios na página {page}")
                    break
                
                print(f"  📄 Processando página {page}/{PAGES} ({len(repos_data['items'])} repos)")
                
                for i, repo in enumerate(repos_data["items"]):
                    print(f"    📊 Processando repo {i+1}/{len(repos_data['items'])}: {repo['name']}")
                    
                    repo_info = {
                        "name": repo["name"],
                        "owner": repo["owner"]["login"],
                        "stars": repo["stargazers_count"],
                        "forks": repo["forks_count"],
                        "language": repo["language"],
                        "created_at": repo["created_at"],
                        "updated_at": repo["updated_at"],
                        "size_kb": repo["size"],
                        "watchers_count": repo["watchers_count"],
                        "open_issues": repo["open_issues_count"]
                    }
                    
                    # Buscar informações do owner
                    owner_info = get_owner_info(repo["owner"]["url"])
                    repo_info.update(owner_info)
                    
                    # Buscar estatísticas detalhadas (VERSÃO CORRIGIDA)
                    stats = get_repo_stats(repo["owner"]["login"], repo["name"])
                    repo_info.update(stats)
                    
                    all_repos.append(repo_info)
                    
                    time.sleep(2)  # Delay entre repos
                
                break  # Sai do retry loop se deu certo
                
            except Exception as e:
                print(f"Erro na página {page}, tentativa {attempt + 1}: {e}")
                if attempt == retry_count - 1:
                    print(f"Falha definitiva na página {page}")
                else:
                    time.sleep(30)  # Aguarda antes de tentar novamente
                continue
        
        time.sleep(5)  # Delay entre páginas
            
    return pd.DataFrame(all_repos)

## 🚀 Execução e Exportação
- Arquivo github_repos_completos.csv com todas as linguagens

- Estrutura padronizada para análise

In [20]:
def main():
    print("🚀 Iniciando coleta de dados do GitHub...")
    print(f"📋 Linguagens: {', '.join(LANGUAGES)}")
    print(f"📄 Páginas por linguagem: {PAGES}")
    print(f"📊 Repos por página: {PER_PAGE}")
    print(f"🎯 Total estimado: {len(LANGUAGES) * PAGES * PER_PAGE} repositórios")
    
    # Verifica se já existem dados
    existing_data = load_existing_data()
    if not existing_data.empty:
        print(f"📁 Encontrados {len(existing_data)} repositórios existentes")
        processed_languages = existing_data['language'].unique().tolist()
        remaining_languages = [lang for lang in LANGUAGES if lang not in processed_languages]
        if remaining_languages:
            print(f"🔄 Continuando com: {', '.join(remaining_languages)}")
            languages_to_process = remaining_languages
        else:
            print("✅ Todas as linguagens já foram processadas!")
            return
    else:
        languages_to_process = LANGUAGES
    
    total_collected = len(existing_data)
    
    for i, lang in enumerate(languages_to_process):
        print(f"\n{'='*60}")
        print(f"🎯 LINGUAGEM {i+1}/{len(languages_to_process)}: {lang}")
        print(f"{'='*60}")
        
        try:
            df = fetch_repos_by_language(lang)
            
            if not df.empty:
                # Determina se é a primeira linguagem NOVA sendo processada
                is_first_new = (i == 0 and existing_data.empty)
                
                save_data_incremental(df, lang, is_first_new)
                total_collected += len(df)
                
                print(f"✅ {len(df)} repositórios de {lang} coletados!")
                print(f"📊 Total acumulado: {total_collected} repositórios")
            else:
                print(f"⚠️  Nenhum repositório coletado para {lang}")
            
            # Delay entre linguagens
            if i < len(languages_to_process) - 1:
                print(f"⏳ Aguardando 5 minutos antes da próxima linguagem...")
                time.sleep(300)
                
        except KeyboardInterrupt:
            print(f"\n⚠️  Interrompido pelo usuário na linguagem {lang}")
            print(f"📊 Dados salvos até agora: {total_collected} repositórios")
            break
        except Exception as e:
            print(f"❌ Erro crítico na linguagem {lang}: {e}")
            continue
    
    # Carrega dados finais
    final_data = load_existing_data()
    if not final_data.empty:
        print(f"\n🎉 COLETA FINALIZADA!")
        print(f"📊 Total final: {len(final_data)} repositórios")
        print(f"📁 Arquivo: {OUTPUT_FILE}")
        print(f"💾 Tamanho: {os.path.getsize(OUTPUT_FILE) / 1024 / 1024:.2f} MB")
        
        # Estatísticas por linguagem
        print(f"\n📈 Distribuição por linguagem:")
        lang_stats = final_data['language'].value_counts()
        for lang, count in lang_stats.items():
            print(f"  {lang}: {count} repositórios")
    else:
        print("❌ Nenhum dado foi coletado.")

if __name__ == "__main__":
    main()

🚀 Iniciando coleta de dados do GitHub...
📋 Linguagens: Python, JavaScript, Java, C#, C++, TypeScript, Go, Rust, Kotlin, Swift
📄 Páginas por linguagem: 15
📊 Repos por página: 100
🎯 Total estimado: 15000 repositórios

🎯 LINGUAGEM 1/10: Python

🔍 Iniciando coleta para Python...
  📄 Processando página 1/15 (100 repos)
    📊 Processando repo 1/100: free-programming-books
    📊 Processando repo 2/100: public-apis
    📊 Processando repo 3/100: system-design-primer
    📊 Processando repo 4/100: awesome-python
    📊 Processando repo 5/100: Python
    📊 Processando repo 6/100: AutoGPT
    📊 Processando repo 7/100: stable-diffusion-webui
    📊 Processando repo 8/100: transformers
    📊 Processando repo 9/100: youtube-dl
    📊 Processando repo 10/100: HelloGitHub
    📊 Processando repo 11/100: yt-dlp
    📊 Processando repo 12/100: DeepSeek-V3
    📊 Processando repo 13/100: thefuck
    📊 Processando repo 14/100: pytorch
    📊 Processando repo 15/100: fastapi
    📊 Processando repo 16/100: django
  

## 📋 Estrutura do Dataset (Colunas Extraídas)

| Coluna                 | Tipo de Dado | Descrição                                                                 | Exemplo                     |
|------------------------|--------------|---------------------------------------------------------------------------|-----------------------------|
| **Informações Básicas** |              |                                                                           |                             |
| `name`                | string       | Nome do repositório                                                      | `tensorflow`                |
| `owner`               | string       | Login do usuário/organização dono                                        | `google`                    |
| `language`            | string       | Linguagem principal do projeto                                           | `Python`                    |
| **Estatísticas**       |              |                                                                           |                             |
| `stars`               | integer      | Número de estrelas                                                       | `175000`                    |
| `forks`               | integer      | Número de forks                                                          | `85000`                     |
| `watchers_count`      | integer      | Usuários acompanhando o repositório                                      | `3200`                      |
| `subscribers_count`   | integer      | Inscritos no repositório (diferente de stars)                            | `1500`                      |
| **Atividade**          |              |                                                                           |                             |
| `open_issues`         | integer      | Issues abertas no momento                                                | `42`                        |
| `closed_issues`       | integer      | Issues fechadas nos últimos 6 meses                                      | `128`                       |
| `pull_requests`       | integer      | Total de PRs (abertos + fechados)                                        | `75`                        |
| `last_year_commits`   | integer      | Commits realizados nos últimos 12 meses                                  | `890`                       |
| `contributors`        | integer      | Número de contribuidores únicos                                          | `35`                        |
| **Metadados**          |              |                                                                           |                             |
| `created_at`          | datetime     | Data de criação do repositório (UTC)                                     | `2015-11-09T23:25:38Z`      |
| `updated_at`          | datetime     | Data da última atualização                                               | `2024-03-15T08:12:45Z`      |
| `size_kb`             | integer      | Tamanho aproximado do repositório em KB                                  | `10240`                     |
| **Informações do Dono**|              |                                                                           |                             |
| `owner_type`          | string       | Tipo do dono (`User` ou `Organization`)                                  | `Organization`              |
| `owner_public_repos`  | integer      | Quantidade de repositórios públicos do dono                              | `250`                       |
| `owner_location`      | string       | Localização geográfica declarada no perfil (opcional)                    | `Mountain View, California` |

## Repositorio

https://github.com/LucasjsSilva/data-set-repositorios